<a href="https://colab.research.google.com/github/SevillaBK/Tensorflow2.0/blob/master/(2)_TF2_tutorial_%EC%BB%A8%EB%B3%BC%EB%A3%A8%EC%85%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로 공식블로그 튜토리얼 : 컨볼루션 신경망
  - https://www.tensorflow.org/tutorials/images/cnn#import_packages

이번 튜토리얼은 운동화, 셔츠, 가방 등 패션 관련 이미지를 분류하는 신경망 모델을 만들어보는 내용입니다.

이번 튜토리얼은 MNIST 숫자를 분류하기 위한 간단한 컨볼루션 신경망 모델을 만들어보는 내용입니다.

In [0]:
# 라이브러리 불러오기
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

## 1. MNIST 데이터셋 임포트하기

In [0]:
# MNIST 데이터셋 불러오기
(train_X, train_Y), (test_X, test_Y) = datasets.mnist.load_data()

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)


# 모델에 넣기 위한 데이터 변환
train_X = train_X.reshape((60000, 28, 28, 1))
test_X = test_X.reshape((10000, 28, 28, 1))
train_X, test_X = train_X / 255.0 , test_X / 255.0

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


훈련 데이터셋은 28x28 픽셀의 60,000개의 데이터로 이루어져 있고,<br/>
테스트 데이터는 28x28 픽셀의 10,000개 데이터로 이루어져 있습니다.

각 픽셀은 0과 255 사이의 값으로 되어 있어 0과 1사이로 스케일링 해주었습니다.

## 2. 합성곱 층(컨볼루션 레이어) 만들기

아래의 코드는 **Conv2D**와 **MaxPooling2D** 층을 쌓는 일반적인 패턴입니다.<br/>
CNN은 배치 크기를 제외하고 (이미지높이, 이미지너비, 컬러채널) 크기의 텐서를 입력으로 받습니다.<br/>
MNIST 데이터는 흑백이미지이므로 컬러 채널이 하나이지만 컬러이미지는 RGB 세개의 채널을 가집니다.

이번 예제에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 만듭니다.

이 값은 첫 번째 층의 `input_shape` 매개변수로 전달됩니다.

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


위에서 Conv2D와 MaxPooling2 층의 출력은 (높이, 너비, 채널) 크기의 3D 텐서입니다. 높이와 너비 차원은 네트워크가 깊어질 수록 감소하는 경향을 가집니다. 

Conv2D 층의 출력 채널의 수는 첫 번째 변수(filters)에 의해 결정됩니다. 일반적으로 높이와 너비가 줄어듦에 따라 Conv2D 출력 채널을 늘릴 수 있습니다.

## 3. 마지막에 Dense 층 추가하기

모델을 완성하려면 마지막 합성곱층의 출력텐서 (3, 3, 64)를 Dense층에 주입하여 분류를 수행해야 합니다. Dense 층은 1차원의 텐서를 입력값으로 받기 때문에 현재 3차원 텐서를 Flatten 층을 이용해 1차원으로 변환해줍니다. <br/><br/>
MNIST 데이터는 10개의 클래스가 있으므로 마지막 Dense 층에 10개의 출력과 softmax 활성화함수를 사용합니다.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

아까의 모델에서 2개의 Dense 층이 추가되었고, 그 앞에 (3, 3, 64)의 출력을 (576) 크기로 변환시켜주는 과정이 있었습니다.

## 4. 모델 컴파일과 훈련하기

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_X, train_Y, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1399 - accuracy: 0.9562
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0434 - accuracy: 0.9865
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0319 - accuracy: 0.9901
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0241 - accuracy: 0.9924
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0207 - accuracy: 0.9935


## 5. 모델 평가

In [0]:
test_loss, test_acc = model.evaluate(test_X, test_Y, verbose=2)

print(test_loss)
print(test_acc)

313/313 - 1s - loss: 0.0344 - accuracy: 0.9899
0.034359160810709
0.9898999929428101


MNIST 데이터셋에 대해 간단한 모델로도 98.9%의 정확도를 달성할 수 있었습니다.